In [19]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    download_loader,
    RAKEKeywordTableIndex,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from dotenv import load_dotenv
load_dotenv()

True

## Load Data + Setup
* default llm = OpenAI(model="gpt-3.5-turbo")
* default embed_model = OpenAIEmbedding()

In [14]:
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4-0613")
Settings.embed_model = OpenAIEmbedding()

In [3]:
"""
1. Data Load: SimpleDirectoryReader
"""

reader = SimpleDirectoryReader(input_files=["../../assets/visa_kor.pdf"])
docs = reader.load_data()

In [3]:
"""
2. Data Load: PDFReader
"""

from llama_index.readers.file import PDFReader
from pathlib import Path

loader = PDFReader()
docs = loader.load_data(file=Path("../../assets/visa_kor.pdf"))

In [4]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs])
docs = [Document(text=doc_text)]

In [58]:
"""
3. Data Load: JSON
"""

from llama_index.readers.json import JSONReader

json_reader = JSONReader()
docs = json_reader.load_data("data/visa_kor_docs.json")

In [5]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode


node_parser = SentenceSplitter(chunk_size=1024)

base_nodes = node_parser.get_nodes_from_documents(docs)

# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

## Baseline Retriever
Define a baseline retriever that simply fetches the top-k raw text nodes by embedding similarity.

In [15]:
base_index = VectorStoreIndex(base_nodes)
base_retriever = base_index.as_retriever(similarity_top_k=2)

retrievals = base_retriever.retrieve(
    "고용허가제 해당 업종 중 서비스업에서 E-9 비자로 취업할 수 있는 세부 업종과 조건은 무엇인가요?"
)

In [16]:
from llama_index.core.response.notebook_utils import display_source_node

for n in retrievals:
    display_source_node(n, source_length=1500)


**Node ID:** node-0<br>**Similarity:** 0.9048427041103281<br>**Text:** E-9:
비전문취업
비자
고용허가제
고용허가제
해당자와
활동범위
고용허가제
해당자에
대한
사증
발급
사증발급
후
1
회
체류기간의
상한
사증발급
허용업종
및
체류자격
약호
(
기호
):
E-9
근로자를
채용할
수
있는
업종과
범위
사증발급인정서를
받아야만
E-9
사증을
발급받을
수
있음
2012.8.1
부터
범죄경력증명서
및
건강상태확인서
제출
비전문취업
(
E-9)
자격
사증발급인정서
발급
절차
재입국특례
제도
(
구
성실근로자
제도
)
재입국특례자에
대한
사증
신청
및
발급
방법
재입국특례자에
대한
우대
내용
고용허가제
해당자의
근무처
(
직장
)
의
변경
고용허가제
해당자의
근무처
(
직장
)
를
변경할
수
있는
조건
고용허가제
해당자의
근무처
(
직장
)
를
변경하는
절차
및
제출
서류
고용허가제
농업
분야
외국인근로자의
근무처
(
직장
)
추가
고용허가제
해당자의
체류자격
변경허가
-
사증
변경
고용허가제
해당자의
체류기간
연장
허가
-
사증
연장
고용허가제
해당자의
재입국허가
고용허가제
해당자의
외국인등록
고용허가제
해당자의
고용변동
신고
고용허가제
●
｢
외국인근로자의
고용
등에
관한
법률
｣
에
의거
,
사업주에게
외국인근로자의
고용을
허가하고
,
외국인
근로자에게는
당해
사업주에게
고용되는
조건으로
최장
4
년
10
개월간
취업을
허용하는
인력제도로
,
‘04.
8
월
제도
시행
이후
현재까지
16
개국과
MOU
를
체결하여
운영
●
상시근로자
(
고용보험
기준
)
300
인
미만
또는
자본금
80
억원
이하
중소기업에
외국인
고용허가
●
고용허가제
선정국가
(16
개국
)
○
태국
,
필리핀
,
스리랑카
,
베트남
,
인도네시아
,
몽골
,
파키스탄
,
우즈베키스탄
,<br>

**Node ID:** node-2<br>**Similarity:** 0.8836058929641023<br>**Text:** 뿌리산업
○
체류자격
:
E-9-1
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
상시근로자
300
인
미만
또는
자본금
80
억원
이하
제조업
■
단
,
상기
기준에
충족하지
않더라도
아래
증빙서류
제출
시
인정
:
중소기업
(
지방중소기업청에서
발급한
‘
중소기업확인서
’)
/
비수도권
소재
뿌리산업
중견기업
(
한국생산기술연구원에서
발급한
‘
뿌리기업
확인서
’
및
한국중견기업연합회에서
발급한
‘
중견기업
확인서
’)
●
건설업
○
체류자격
:
E-9-2
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
모든
건설공사
■
※
발전소
·
제철소
·
석유화학
건설현장의
건설업체
중
건설면허가
산업환경
설비인
경우에는
적용
제외
●
농축산업
○
체류자격
:
E-9-3
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
농업
:
작물재배업
■
축산업
■
농축산서비스업
:
작물재배
및
축산관련
서비스업
●
어업
○
체류자격
:
E-9-4
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)

■
연근해어업
:
연안어업ㆍ근해어업
●
선원법
적용을
받지
않는
20
톤
미만
어선
종사자에
한함
■
양식어업
■
소금채취업
●
임업
○
체류자격
:
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
임업
종묘
생산업
,
육림업
,
벌목업
,<br>

In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine

llm = OpenAI(model="gpt-4-0613")
query_engine_base = RetrieverQueryEngine.from_args(retriever=base_retriever, llm=llm, streaming=True)

response = query_engine_base.query(
    "비전문취업 비자의 일반적인 허용 업종 범위는 어디까지인가요?"
    # "고용허가제 해당 업종 중 서비스업에서 E-9 비자로 취업할 수 있는 세부 업종과 조건은 무엇인가요?"
)
response.print_response_stream()


비전문취업 비자(E-9)의 허용 업종 범위에 대한 구체적인 정보는 제공된 문맥에서 확인할 수 없습니다.

In [23]:
for node in response.source_nodes:
    print("-----")
    text_fmt = node.node.get_content().strip().replace("\n", " ")[:1000]
    print(f"Text:\t {text_fmt} ...")
    print(f"Metadata:\t {node.node.metadata}")
    print(f"Score:\t {node.score:.3f}")

-----
Text:	 비전문취업자의 근무처 변경에 대한 허가권자는 법무부장관임 ● 근무처 ( 직장 ) 변경 ( 이동 ) 의 제한 ○ 비전문취업자는 원칙적으로 최초 고용허가를 받은 사업장에서 계속 취업하여야 함 ○ 다만 , 예외적으로 사업장의 휴 ․ 폐업 등 그 사업 또는 사업장에서 정상적인 근로관계를 지속하기 곤란한 때에는 다른 사업 또는 사업장으로의 변경 신청 가능 ● 변경 횟수의 제한 ○ 비전문취업자의 사업장 변경은 입국한 날부터 3 년 이내의 취업가능기간 중에는 원칙적으로 3 회 , 재고용절차에 따라 체류기간이 연장된 기간 중에는 2 회를 초과할 수 없음 ○ 다만 , 휴 ․ 폐업 , ...
Metadata:	 {}
Score:	 0.851
-----
Text:	 E-9: 비전문취업 비자 고용허가제 고용허가제 해당자와 활동범위 고용허가제 해당자에 대한 사증 발급 사증발급 후 1 회 체류기간의 상한 사증발급 허용업종 및 체류자격 약호 ( 기호 ): E-9 근로자를 채용할 수 있는 업종과 범위 사증발급인정서를 받아야만 E-9 사증을 발급받을 수 있음 2012.8.1 부터 범죄경력증명서 및 건강상태확인서 제출 비전문취업 ( E-9) 자격 사증발급인정서 발급 절차 재입국특례 제도 ( 구 성실근로자 제도 ) 재입국특례자에 대한 사증 신청 및 발급 방법 재입국특례자에 대한 우대 내용 고용허가제 해당자의 근무처 ( 직장 ) 의 변경 고용허가제 해당자의 근무처 ( 직장 ) 를 변경할 수 있는 조건 고용허가제 해당자의 근무처 ( 직장 ) 를 변경하는 절차 및 제출 서류 고용허가제 농업 분야 외국인근로자의 근무처 ( 직장 ) 추가 고용허가제 해당자의 체류자격 변경허가 - 사증 변경 고용허가제 해당자의 체류기간 연장 허가 - 사증 연장 고용허가제 해당자의 재입국허가 고용허가제 해당자의 외국인등록 고용허가제 해당자의 고용변동 신고 고용허가제 ● ｢ 외국인근로자의 고용 등에 관한 법률 ｣ 에 의거 , 사업주에게 외국인근로자의 고용을 허가하고 , 외국인 근로자에게는 당해 사업주에

## 메타데이터 참조: 요약 + 더 큰 청크를 참조하는 생성된 질문 
이 사용 예에서는 소스 노드를 참조하는 추가 컨텍스트를 정의하는 방법을 보여줍니다.

이 추가 컨텍스트에는 생성된 질문뿐만 아니라 요약도 포함됩니다.

쿼리 시간 동안 우리는 더 작은 청크를 검색하지만 더 큰 청크에 대한 참조를 따릅니다. 이를 통해 우리는 합성에 대한 더 많은 맥락을 가질 수 있습니다.

In [24]:
import nest_asyncio

nest_asyncio.apply()


from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

extractors = [
    SummaryExtractor(summaries=["self"], show_progress=True),
    QuestionsAnsweredExtractor(questions=5, show_progress=True),
]

In [25]:
# run metadata extractor across base nodes, get back dictionaries
node_to_metadata = {}
for extractor in extractors:
    metadata_dicts = extractor.extract(base_nodes)
    for node, metadata in zip(base_nodes, metadata_dicts):
        if node.node_id not in node_to_metadata:
            node_to_metadata[node.node_id] = metadata
        else:
            node_to_metadata[node.node_id].update(metadata)

100%|██████████| 20/20 [00:29<00:00,  1.46s/it]


In [27]:
import json

# cache metadata dicts
def save_metadata_dicts(path, data):
    with open(path, "w") as fp:
        json.dump(data, fp)


def load_metadata_dicts(path):
    with open(path, "r") as fp:
        data = json.load(fp)
    return data

In [29]:
save_metadata_dicts("data/visa_kor_metadata.json", node_to_metadata)

In [32]:
metadata_dicts = load_metadata_dicts("data/visa_kor_metadata.json")

In [33]:
# all nodes consists of source nodes, along with metadata
import copy

all_nodes = copy.deepcopy(base_nodes)
for node_id, metadata in node_to_metadata.items():
    for val in metadata.values():
        all_nodes.append(IndexNode(text=val, index_id=node_id))

all_nodes_dict = {n.node_id: n for n in all_nodes}

In [49]:
vector_index_chunk = VectorStoreIndex(all_nodes, embed_model=OpenAIEmbedding())
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [35]:
vector_index_metadata = VectorStoreIndex(all_nodes)
vector_retriever_metadata = vector_index_metadata.as_retriever(
    similarity_top_k=2
)

In [42]:
retrievals = vector_retriever_metadata.retrieve("고용허가제 해당 업종 중 서비스업에서 E-9 비자로 취업할 수 있는 세부 업종과 조건은 무엇인가요?")

for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-0<br>**Similarity:** 0.904839902930188<br>**Text:** E-9:
비전문취업
비자
고용허가제
고용허가제
해당자와
활동범위
고용허가제
해당자에
대한
사증
발급
사증발급
후
1
회
체류기간의
상한
사증발급
허용업종
및
체류자격
약호
(
기호
):
E-9
근로자를
채용할
수
있는
업종과
범위
사증발급인정서를
받아야만
E-9
사증을
발급받을
수
있음
2012.8.1
부터
범죄경력증명서
및
건강상태확인서
제출
비전문취업
(
E-9)
자격
사증발급인정서
발급
절차
재입국특례
제도
(
구
성실근로자
제도
)
재입국특례자에
대한
사증
신청
및
발급
방법
재입국특례자에
대한
우대
내용
고용허가제
해당자의
근무처
(
직장
)
의
변경
고용허가제
해당자의
근무처
(
직장
)
를
변경할
수
있는
조건
고용허가제
해당자의
근무처
(
직장
)
를
변경하는
절차
및
제출
서류
고용허가제
농업
분야
외국인근로자의
근무처
(
직장
)
추가
고용허가제
해당자의
체류자격
변경허가
-
사증
변경
고용허가제
해당자의
체류기간
연장
허가
-
사증
연장
고용허가제
해당자의
재입국허가
고용허가제
해당자의
외국인등록
고용허가제
해당자의
고용변동
신고
고용허가제
●
｢
외국인근로자의
고용
등에
관한
법률
｣
에
의거
,
사업주에게
외국인근로자의
고용을
허가하고
,
외국인
근로자에게는
당해
사업주에게
고용되는
조건으로
최장
4
년
10
개월간
취업을
허용하는
인력제도로
,
‘04.
8
월
제도
시행
이후
현재까지
16
개국과
MOU
를
체결하여
운영
●
상시근로자
(
고용보험
기준
)
300
인
미만
또는
자본금
80
억원
이하
중소기업에
외국인
고용허가
●
고용허가제
선정국가
(16
개국
)
○
태국
,
필리핀
,
스리랑카
,
베트남
,
인도네시아
,
몽골
,
파키스탄
,
우즈베키스탄
,<br>

**Node ID:** node-2<br>**Similarity:** 0.8835613442412317<br>**Text:** 뿌리산업
○
체류자격
:
E-9-1
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
상시근로자
300
인
미만
또는
자본금
80
억원
이하
제조업
■
단
,
상기
기준에
충족하지
않더라도
아래
증빙서류
제출
시
인정
:
중소기업
(
지방중소기업청에서
발급한
‘
중소기업확인서
’)
/
비수도권
소재
뿌리산업
중견기업
(
한국생산기술연구원에서
발급한
‘
뿌리기업
확인서
’
및
한국중견기업연합회에서
발급한
‘
중견기업
확인서
’)
●
건설업
○
체류자격
:
E-9-2
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
모든
건설공사
■
※
발전소
·
제철소
·
석유화학
건설현장의
건설업체
중
건설면허가
산업환경
설비인
경우에는
적용
제외
●
농축산업
○
체류자격
:
E-9-3
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
농업
:
작물재배업
■
축산업
■
농축산서비스업
:
작물재배
및
축산관련
서비스업
●
어업
○
체류자격
:
E-9-4
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)

■
연근해어업
:
연안어업ㆍ근해어업
●
선원법
적용을
받지
않는
20
톤
미만
어선
종사자에
한함
■
양식어업
■
소금채취업
●
임업
○
체류자격
:
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
임업
종묘
생산업
,
육림업
,
벌목업
,<br>

## 평가 
재귀 검색 + 노드 참조 방법이 얼마나 잘 작동하는지 평가합니다. 우리는 청크 참조와 메타데이터 참조를 모두 평가합니다. 참조 노드를 검색하기 위해 임베딩 유사성 조회를 사용합니다.

우리는 원시 노드를 직접 가져오는 기본 검색기와 두 방법을 비교합니다.

지표 측면에서는 적중률과 MRR을 모두 사용하여 평가합니다.

In [43]:
"""
데이터세트 생성 
먼저 텍스트 덩어리 세트에서 질문 데이터세트를 생성합니다.
"""

from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.llms.openai import OpenAI

import nest_asyncio

nest_asyncio.apply()


eval_dataset = generate_question_context_pairs(
    base_nodes, OpenAI(model="gpt-4-0613")
)

100%|██████████| 20/20 [01:13<00:00,  3.67s/it]


In [45]:
eval_dataset.save_json("data/visa_kor_dataset.json")

In [47]:
eval_dataset = EmbeddingQAFinetuneDataset.from_json(
    "data/visa_kor_dataset.json"
)

## 결과 비교 
적중률과 MRR을 측정하기 위해 각 검색기에 대한 평가를 실행합니다.

우리는 노드 참조(청크 또는 메타데이터)가 있는 검색기가 원시 청크를 검색하는 것보다 더 나은 성능을 보이는 경향이 있음을 발견했습니다.

In [48]:
import pandas as pd
from llama_index.core.evaluation import (
    RetrieverEvaluator,
    get_retrieval_results_df,
)

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)

In [50]:
from llama_index.core.retrievers import RecursiveRetriever

vector_retriever_chunk = vector_index_chunk.as_retriever(
    similarity_top_k=top_k
)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)
# try it out on an entire dataset
results_chunk = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/40 [00:00<?, ?it/s]

Retrieving with query id None: "What is the 'Re-entry Special System' (also known as the Sincere Worker System) and what are the conditions for a foreign worker to be eligible for this system according to the context provided?"
Retrieved node with id, entering: node-8
Retrieving with query id node-8: "What is the 'Re-entry Special System' (also known as the Sincere Worker System) and what are the conditions for a foreign worker to be eligible for this system according to the context provided?"
Retrieved node with id, entering: node-10
Retrieving with query id node-10: "What is the 'Re-entry Special System' (also known as the Sincere Worker System) and what are the conditions for a foreign worker to be eligible for this system according to the context provided?"
Retrieved node with id, entering: node-17
Retrieving with query id node-17: "What is the 'Re-entry Special System' (also known as the Sincere Worker System) and what are the conditions for a foreign worker to be eligible for thi

100%|██████████| 40/40 [00:12<00:00,  3.23it/s]

Retrieving text node: 필리핀
,
스리랑카
,
베트남
,
인도네시아
,
몽골
,
파키스탄
,
우즈베키스탄
,
캄보디아
,
중국
,
방글라데시
,
네팔
,
미얀마
,
키르기스스탄
,
동티모르
,
라오스

고용허가제
해당자와
활동범위
●
활동범위
○
외국인근로자의
고용
등에
관한
법률의
규정에
의한
국내취업
●
해당자
○
외국인근로자의
고용
등에
관한
법률의
규정에
의한
국내
취업요건을
갖춘
자
(
다만
,
일정
자격이나
경력
등이
필요한
전문직종에
종사하고자
하는
자는
제외
)
고용허가제
해당자에
대한
사증
발급
사증발급
후
1
회
체류기간의
상한
●
3
년
사증발급
허용업종
및
체류자격
약호
(
기호
):
E-9
근로자를
채용할
수
있는
업종과
범위
●
제조업
:
제조업
,
뿌리산업
○
체류자격
:
E-9-1
○
적용범위
(
E-9
근로자를
고용할
수
있는
조건
)
■
상시근로자
300
인
미만
또는
자본금
80
억원
이하
제조업
■
단
,
Retrieving text node: 【
붙임
3
】
참조
)
고용허가제
농업
분야
외국인근로자의
근무처
(
직장
)
추가
●
계절적으로
업무량의
차이가
큰
일부
농업분야에
근무하는
외국인근로자가
원
사업장과의
근로계약은
유지하면서
(
무급휴직
처리
)
일정기간
동안
다른
사업주와
근로계약을
체결하여
근무한
후
,
근로계약기간이
만료되면
원
사업장으로
복귀하도록
하는
제도
●
신청
절차
○
신청자격
:
작물재배업
(
계절적
농업
)
에
근무
중인
비전문취업
(
E-9-3)
자격자
(
농협
대행
가능
)
○
신청방법
:
원
근무처
관할
출입국ㆍ외국인청
(
사무소ㆍ출장소
)
방문
접수
(
농협
대행
가능
)
●
고용주
신고의무
(
농업분야
)
○
출입국관리법
제
19
조의
고용주
신고의무는
원
근무처
고용주와
추가
근무처
고용주에게
모두
적용
■
단
,
정상적인
절차로
복귀하는
경우
원
근무처
고용주
신고의무
면제
○
근무처
추가
허가
시
부터
근로계약
종료로
복귀하기
전까지
해고
,
이탈
,
퇴직
등의
사유

In [51]:
vector_retriever_metadata = vector_index_metadata.as_retriever(
    similarity_top_k=top_k
)
retriever_metadata = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_metadata},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_metadata
)
# try it out on an entire dataset
results_metadata = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/40 [00:00<?, ?it/s]

Retrieving with query id None: What are the requirements for the criminal record certificate that needs to be submitted for the E-9 visa application, and what can be done if the system for checking criminal records in the applicant's home country is insufficient?
Retrieved node with id, entering: node-5
Retrieving with query id node-5: What are the requirements for the criminal record certificate that needs to be submitted for the E-9 visa application, and what can be done if the system for checking criminal records in the applicant's home country is insufficient?
Retrieved node with id, entering: node-0
Retrieving with query id node-0: What are the requirements for the criminal record certificate that needs to be submitted for the E-9 visa application, and what can be done if the system for checking criminal records in the applicant's home country is insufficient?
Retrieved node with id, entering: node-16
Retrieving with query id node-16: What are the requirements for the criminal rec

100%|██████████| 40/40 [00:11<00:00,  3.40it/s]

Retrieved node with id, entering: node-6
Retrieving with query id node-6: "What are the specific additional documents required for visa issuance recognition in the fishing industry and aquaculture industry as per the guidelines on visa.go.kr?"
Retrieved node with id, entering: node-7
Retrieving with query id node-7: "What are the specific additional documents required for visa issuance recognition in the fishing industry and aquaculture industry as per the guidelines on visa.go.kr?"
Retrieved node with id, entering: node-18
Retrieving with query id node-18: "What are the specific additional documents required for visa issuance recognition in the fishing industry and aquaculture industry as per the guidelines on visa.go.kr?"
Retrieved node with id, entering: node-8
Retrieving with query id node-8: "What are the specific additional documents required for visa issuance recognition in the fishing industry and aquaculture industry as per the guidelines on visa.go.kr?"
Retrieved node with id

In [52]:
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)
# try it out on an entire dataset
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

100%|██████████| 40/40 [00:05<00:00,  7.01it/s]


In [53]:
full_results_df = get_retrieval_results_df(
    [
        "Base Retriever",
        "Retriever (Chunk References)",
        "Retriever (Metadata References)",
    ],
    [results_base, results_chunk, results_metadata],
)
display(full_results_df)

,retrievers,hit_rate,mrr
0,Base Retriever,0.975,0.594881
1,Retriever (Chunk References),0.975,0.818750
2,Retriever (Metadata References),0.975,0.818750
